In [1]:
# Input info
# se_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
# minCell = 1
# outdir = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/clone_counts/minCellConds_1" 
# sample_names = "pre,post"

se_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/SE.rds"
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/minCellConds_1"
sample_names = "Control,Flt3l,Input" 
minCell = 1 


# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"



In [2]:
source(utils_script)
#source("/data/Mito_Trace/R_scripts/plot_de_over_all.R")

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
dir.create(outdir, showWarnings = TRUE)

Warning message in dir.create(outdir, showWarnings = TRUE):
“'/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/minCellConds_1' already exists”


In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "Control" "Flt3l"   "Input"

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package:

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
library(dplyr)

In [7]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
221404 features across 18672 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

# Get clones with mininum  cells in each condition and run DE pre and post

In [8]:
source(utils_script)
tmp = get.clone.sizes(se)
counts.norm=tmp[["counts.norm"]]
clone.sizes = tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the
`.groups` argument.
Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.


donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,0,0.26597520,753,0_0,0.1329876,753,1
0,1,0.17964767,522,0_1,0.2228114,1275,2
0,2,0.16247703,480,0_2,0.3040499,1755,3
0,53,0.10094447,283,0_53,0.3545222,2038,4
0,4,0.08522127,246,0_4,0.3971328,2284,5
0,3,0.08504011,262,0_3,0.4396529,2546,6


In [22]:
curr.counts.norm = counts.norm[counts.norm$size>minCell,]
#tmp.thresh = curr.counts.norm %>% group_by(name) %>% summarize(size=sum(size))
#curr.counts.norm = curr.counts.norm[tmp.thresh$size > 1, ]

curr.counts.norm = curr.counts.norm[order(curr.counts.norm$name),]
topClones = unique(curr.counts.norm$name)
#curr.counts.norm

## Save

In [28]:
out.curr.counts.norm = tidyr::pivot_wider(curr.counts.norm, id_cols="name",values_from="norm", names_from="condition")
out.curr.counts.norm$size.mean = rowMeans(out.curr.counts.norm %>% dplyr::select(dplyr::all_of(sample_names)))



In [29]:

if(length(sample_names)==1){
    out.curr.counts.norm$fold = 1
}else{
    out.curr.counts.norm$fold =  out.curr.counts.norm[[sample_names[1]]]/out.curr.counts.norm[[sample_names[2]]]
    #out.curr.counts.norm = out.curr.counts.norm %>% arrange(desc(size.mean)) %>% dplyr::select(-c(size.mean))
    }
out.curr.counts.norm = out.curr.counts.norm %>% arrange(desc(fold))  %>% dplyr::select(-c(size.mean))
ord.inds = as.character(out.curr.counts.norm[["name"]])
out.curr.counts.norm



name,Control,Flt3l,Input,fold
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1_24,0.021242485,0.0061079063,0.0299566417,3.477867
1_7,0.027655311,0.0091618595,0.0516357903,3.018526
1_5,0.036072144,0.0122158127,0.0441466299,2.952906
1_19,0.020440882,0.0071258907,0.0366574695,2.868537
1_14,0.025250501,0.0091618595,0.0374458021,2.756045
0_56,0.001532959,0.0006079027,0.0008789921,2.521717
1_6,0.027655311,0.0118764846,0.0500591250,2.328577
1_22,0.025250501,0.0108585002,0.0220733149,2.325413
1_48,0.004408818,0.0020359688,0.0094599921,2.165464


In [31]:
out.curr.counts = tidyr::pivot_wider(curr.counts.norm, id_cols="name",values_from="size", names_from="condition")
out.curr.counts = as.data.frame(out.curr.counts)
row.names(out.curr.counts) = out.curr.counts$name
out.curr.counts = out.curr.counts[ord.inds,]
out.curr.counts

,name,Control,Flt3l,Input
,<fct>,<int>,<int>,<int>
1_24,1_24,53,18,76
1_7,1_7,69,27,131
1_5,1_5,90,36,112
1_19,1_19,51,21,93
1_14,1_14,63,27,95
0_56,0_56,3,2,3
1_6,1_6,69,35,127
1_22,1_22,63,32,56
1_48,1_48,11,6,24


In [32]:
readr::write_csv(out.curr.counts.norm, file.path(outdir, "clone_sizes_norm.csv"))
readr::write_csv(out.curr.counts, file.path(outdir, "clone_sizes.csv"))

In [ ]:
ggplot(data=curr.counts.norm, aes(x=name,y=norm, fill = condition)) + 
        geom_col(position='dodge') 
    

ggsave(file.path(outdir, "topClonesSize_NormBatch.pdf"))

Saving 6.67 x 6.67 in image



In [ ]:
ggplot(data=curr.counts.norm, aes(x=name,y=size, fill = condition)) + 
        geom_col(position='dodge') 
    

ggsave(file.path(outdir, "topClonesSize.pdf"))

In [ ]:
sessionInfo()